# Pedigree Scraper (Lightweight)

- **目的**: 血統データのみを取得（父・母・BMS）
- **モード**: JRA または NAR を選択
- **特徴**:
  - 過去走データは History Generator で生成するため不要
  - ユニークな horse_id のみを対象（重複スキップ）
  - 1馬あたり1リクエストのみ
- **出力**: `database_xxx_pedigree.csv`


In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ========================================
# ⚙️ モード設定 - ここを変更してください
# ========================================

MODE = 'JRA'  # 'JRA' または 'NAR' を選択

# ========================================

In [ ]:
# Settings (モードに基づいて自動設定)
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'

if MODE == 'JRA':
    BASIC_CSV = 'database_jra_basic.csv'
    PEDIGREE_CSV = 'database_jra_pedigree.csv'
elif MODE == 'NAR':
    BASIC_CSV = 'database_nar_basic.csv'
    PEDIGREE_CSV = 'database_nar_pedigree.csv'
else:
    raise ValueError(f"Invalid MODE: {MODE}. Use 'JRA' or 'NAR'")

print(f"📋 Mode: {MODE}")
print(f"   Input: {BASIC_CSV}")
print(f"   Output: {PEDIGREE_CSV}")

In [ ]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from tqdm.auto import tqdm

print("✅ Libraries loaded")

In [ ]:
# 血統取得関数

def get_pedigree(horse_id):
    """
    馬の血統データのみを取得
    Returns: dict with horse_id, father, mother, bms
    """
    result = {
        'horse_id': horse_id,
        'father': '',
        'mother': '',
        'bms': ''
    }
    
    try:
        time.sleep(0.5)  # Rate limiting
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'}
        
        url = f"https://db.netkeiba.com/horse/ped/{horse_id}/"
        resp = requests.get(url, headers=headers, timeout=15)
        resp.encoding = 'EUC-JP'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        blood_tbl = soup.select_one('table.blood_table')
        if blood_tbl:
            trs = blood_tbl.find_all('tr')
            
            # Father: Row 0, Column 0
            if len(trs) > 0:
                tds0 = trs[0].find_all('td')
                if len(tds0) > 0:
                    result['father'] = tds0[0].text.strip().split('\n')[0]
            
            # Mother: Row at midpoint, Column 0
            # BMS: Row at midpoint, Column 1
            mid_idx = len(trs) // 2
            if mid_idx > 0 and len(trs) > mid_idx:
                tds_m = trs[mid_idx].find_all('td')
                if len(tds_m) > 0:
                    result['mother'] = tds_m[0].text.strip().split('\n')[0]
                if len(tds_m) > 1:
                    result['bms'] = tds_m[1].text.strip().split('\n')[0]
    
    except Exception as e:
        print(f"  ⚠️ Error ({horse_id}): {e}")
    
    return result

print("✅ Pedigree function loaded")

In [ ]:
# メインスクレイピング実行

def run_pedigree_scraping():
    """
    BasicデータからユニークなHorse IDを取得し、血統のみをスクレイピング
    """
    basic_path = os.path.join(SAVE_DIR, BASIC_CSV)
    pedigree_path = os.path.join(SAVE_DIR, PEDIGREE_CSV)
    
    # Load Basic data
    if not os.path.exists(basic_path):
        print(f"❌ Basic file not found: {basic_path}")
        return
    
    df_basic = pd.read_csv(basic_path, dtype=str, usecols=['horse_id'])
    all_horse_ids = set(df_basic['horse_id'].dropna().unique())
    print(f"📋 Unique horses in {MODE} Basic: {len(all_horse_ids)}")
    
    # Load existing pedigree data
    existing_ids = set()
    if os.path.exists(pedigree_path):
        df_existing = pd.read_csv(pedigree_path, dtype=str)
        if 'horse_id' in df_existing.columns and 'father' in df_existing.columns:
            # Only consider complete records
            completed = df_existing[df_existing['father'].notna() & (df_existing['father'] != '')]
            existing_ids = set(completed['horse_id'].unique())
        print(f"💾 Already scraped: {len(existing_ids)}")
    
    # Calculate diff
    target_ids = list(all_horse_ids - existing_ids)
    print(f"🚀 To scrape: {len(target_ids)}\n")
    
    if not target_ids:
        print(f"✅ All {MODE} pedigree data already collected!")
        return
    
    # Scrape with buffer
    buffer = []
    chunk_size = 100
    success_count = 0
    
    for i, horse_id in enumerate(tqdm(target_ids, desc=f"{MODE} Pedigree")):
        pedigree = get_pedigree(horse_id)
        buffer.append(pedigree)
        
        if pedigree['father']:
            success_count += 1
        
        # Circuit Breaker
        if len(buffer) >= 20:
            recent_success = sum(1 for p in buffer[-20:] if p['father'])
            if recent_success < 4:  # < 20% success
                print(f"\n⚠️ CIRCUIT BREAKER: Only {recent_success}/20 successful. Possible IP block.")
                print("⛔ Stopping to prevent empty data accumulation.")
                break
        
        # Save chunk
        if len(buffer) >= chunk_size or i == len(target_ids) - 1:
            df_chunk = pd.DataFrame(buffer)
            
            if not os.path.exists(pedigree_path):
                df_chunk.to_csv(pedigree_path, index=False)
            else:
                df_chunk.to_csv(pedigree_path, mode='a', header=False, index=False)
            
            print(f"  💾 Saved {len(buffer)} pedigrees (success: {sum(1 for p in buffer if p['father'])})")
            buffer = []
    
    print(f"\n✅ {MODE} Pedigree scraping complete!")
    print(f"   Total success: {success_count}")

print("✅ Main function ready")

In [ ]:
# 実行
run_pedigree_scraping()